In [398]:
import pandas as pd
import numpy as np
from itertools import product,chain
from paramsearch import paramsearch
import pandas_profiling as pp

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import matthews_corrcoef
from scipy.stats import rankdata

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import catboost as cb
from sklearn.pipeline import Pipeline

In [12]:
train = pd.read_csv('train.csv', sep = '\t', index_col = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t', index_col = 'Unnamed: 0')

train.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True,axis = 1)
test.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True, axis = 1)


X_train_old, y_train_old = train[train.columns[1:]], train['0']
X_test_old, y_test_old = test[test.columns[1:]], test['0'].fillna(0, inplace = True)




# Bagging on log-reg
best parameters were previously computed

In [56]:
lg = LogisticRegression(class_weight = 'balanced', C = 0.052499)
bg = BaggingClassifier(lg, random_state=42, n_estimators = 5 ,max_samples = 0.9, max_features = 1.0)
bg.fit(X_train_old, y_train_old)



BaggingClassifier(base_estimator=LogisticRegression(C=0.052499, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.9, n_estimators=5, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False)

In [57]:
y_bg = bg.predict(X_test_old)

# random forest

In [59]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1 , class_weight = 'balanced', max_features = 100, n_estimators = 500, max_depth=90, min_samples_leaf= 1)
rfc.fit(X_train_old, y_train_old)

rfc.fit(X_train_old, y_train_old)
y_rfc = rfc.predict(X_test_old)


# ExtraTrees

In [29]:
skf = StratifiedKFold(n_splits=4, random_state=42)

parameters = {'max_features': [17, 30, 35], 'min_samples_leaf': [5, 7, 10], 'max_depth': [15, 20]}
etc = ExtraTreesClassifier(n_estimators=150, random_state=42, n_jobs=-1, class_weight = 'balanced')
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=2, scoring='roc_auc')
etcv.fit(X_train_old, y_train_old)

Fitting 4 folds for each of 18 candidates, totalling 72 fits
[CV] max_depth=15, max_features=17, min_samples_leaf=5 ...............
[CV] max_depth=15, max_features=17, min_samples_leaf=5 ...............
[CV] max_depth=15, max_features=17, min_samples_leaf=5 ...............
[CV] max_depth=15, max_features=17, min_samples_leaf=5 ...............
[CV]  max_depth=15, max_features=17, min_samples_leaf=5, total=  19.4s
[CV] max_depth=15, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=15, max_features=17, min_samples_leaf=5, total=  20.4s
[CV] max_depth=15, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=15, max_features=17, min_samples_leaf=5, total=  21.0s
[CV] max_depth=15, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=15, max_features=17, min_samples_leaf=5, total=  22.0s
[CV] max_depth=15, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=15, max_features=17, min_samples_leaf=7, total=  18.7s
[CV] max_depth=1

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.9min


[CV]  max_depth=15, max_features=35, min_samples_leaf=10, total=  38.3s
[CV] max_depth=20, max_features=17, min_samples_leaf=5 ...............
[CV]  max_depth=15, max_features=35, min_samples_leaf=10, total=  37.3s
[CV] max_depth=20, max_features=17, min_samples_leaf=5 ...............
[CV]  max_depth=15, max_features=35, min_samples_leaf=10, total=  37.2s
[CV] max_depth=20, max_features=17, min_samples_leaf=5 ...............
[CV]  max_depth=20, max_features=17, min_samples_leaf=5, total=  27.5s
[CV] max_depth=20, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=20, max_features=17, min_samples_leaf=5, total=  28.3s
[CV] max_depth=20, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=20, max_features=17, min_samples_leaf=5, total=  28.4s
[CV] max_depth=20, max_features=17, min_samples_leaf=7 ...............
[CV]  max_depth=20, max_features=17, min_samples_leaf=5, total=  29.2s
[CV] max_depth=20, max_features=17, min_samples_leaf=7 ...............
[CV

[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 10.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=False),
       error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=150, n_jobs=-1, oob_score=False, random_state=42,
           verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [15, 20], 'max_features': [17, 30, 35], 'min_samples_leaf': [5, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [30]:
etcv.best_estimator_, etcv.best_score_

(ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=20, max_features=30,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=7,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False), 0.737687976105537)

In [31]:
best_xrf = etcv.best_estimator_
best_xrf.fit(X_train_old, y_train_old)
y_test = best_xrf.predict(X_test_old)

#header = ['_VAL_']
#test1 = test['0'].to_csv('bag_2.csv',sep =',', header = header, index_label = ['_ID_']).values 

print(np.sum(y_test)/len(y_test))
#print(np.sum(np.abs(y_test - test1))/len(y_test))

0.2525204032645223


In [32]:
xrf = etcv.best_estimator_
y_xrf = best_xrf.predict(X_test_old)

# XGBoost

In [47]:
spw = (len(y_train_old) - np.sum(y_train_old))/np.sum(y_train_old)
gbm = xgb.XGBClassifier(n_estimators = 150, subsample=0.8, colsample_bytree=0.8, objective = 'binary:logistic', scale_pos_weight = spw)

dmatrix = xgb.DMatrix(data=X_train_old, label=y_train_old)

#spw = (len(y_train_old) - np.sum(y_train_old))/np.sum(y_train_old)

parameters = {'learning_rate' : [0.1, 0.05], 'max_depth' : [5, 10], 'reg_lambda' : [1]}
gbmcv = GridSearchCV(estimator = gbm, param_grid=parameters, scoring = 'roc_auc', cv = skf, verbose = 2, n_jobs = -1)
gbmcv.fit(X_train_old, y_train_old)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] max_depth=5, reg_lambda=1, learning_rate=0.1 ....................
[CV] max_depth=5, reg_lambda=1, learning_rate=0.1 ....................
[CV] max_depth=5, reg_lambda=1, learning_rate=0.1 ....................
[CV] max_depth=5, reg_lambda=1, learning_rate=0.1 ....................
[CV] ..... max_depth=5, reg_lambda=1, learning_rate=0.1, total= 1.4min
[CV] max_depth=10, reg_lambda=1, learning_rate=0.1 ...................
[CV] ..... max_depth=5, reg_lambda=1, learning_rate=0.1, total= 1.4min
[CV] ..... max_depth=5, reg_lambda=1, learning_rate=0.1, total= 1.4min
[CV] max_depth=10, reg_lambda=1, learning_rate=0.1 ...................
[CV] ..... max_depth=5, reg_lambda=1, learning_rate=0.1, total= 1.4min
[CV] max_depth=10, reg_lambda=1, learning_rate=0.1 ...................
[CV] max_depth=10, reg_lambda=1, learning_rate=0.1 ...................
[CV] .... max_depth=10, reg_lambda=1, learning_rate=0.1, total= 2.5min
[CV] max_depth=5,

[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  7.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=False),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4.624193250968099, seed=0, silent=True,
       subsample=0.8),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [5, 10], 'reg_lambda': [1], 'learning_rate': [0.1, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [48]:
gbmcv.best_estimator_, gbmcv.best_score_

(XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
        gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=5,
        min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=4.624193250968099, seed=0, silent=True,
        subsample=0.8), 0.7384816015468296)

In [49]:
best_gbm = gbmcv.best_estimator_
best_gbm.fit(X_train_old, y_train_old)
y_test = best_gbm.predict(X_test_old)

#header = ['_VAL_']
#test1 = test['0'].to_csv('bag_2.csv',sep =',', header = header, index_label = ['_ID_']).values 

print(np.sum(y_test)/len(y_test))
#print(np.sum(np.abs(y_test - test1))/len(y_test))

0.3170907345175228


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
y_gbm = best_gbm.predict(X_test_old)


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Voting 1
majority

In [197]:
arr = y_bg + y_rfc + y_xrf + y_gbm

In [198]:
arr = np.where(arr < 2 , 0, arr)
arr = np.where(arr >= 2 , 1, arr)

np.sum(arr)/len(arr)

0.3050888142102736

In [199]:
test['0'] = arr
header = ['_VAL_']
test['0'].to_csv('ensemble_4.csv',sep =',', header = header, index_label = ['_ID_']) 

# Voting 2 ( Bagging, Extra trees, Random forest, Xgboost)
rank

CV -score 0.7457
LB -score 0.691

In [110]:
y_gbm_pr = best_gbm.predict_proba(X_test_old)
y_xrf_pr = best_xrf.predict_proba(X_test_old)
y_rfc_pr = rfc.predict_proba(X_test_old)
y_bg_pr = bg.predict_proba(X_test_old)

In [130]:
rfc_predictions = []
gbm_predictions = []
xrf_predictions = []
bg_predictions = []


for train, val in skf.split(X_train_old, y_train_old):
    best_gbm.fit(X_train_old.iloc[train], y_train_old[train])
    best_xrf.fit(X_train_old.iloc[train], y_train_old[train])
    rfc.fit(X_train_old.iloc[train], y_train_old[train])
    bg.fit(X_train_old.iloc[train], y_train_old[train])
    
    
    bg_predictions.append([y_train_old[val], bg.predict_proba(X_train_old.iloc[val])[:,1]])
    rfc_predictions.append([y_train_old[val], rfc.predict_proba(X_train_old.iloc[val])[:,1]])
    gbm_predictions.append([y_train_old[val], best_gbm.predict_proba(X_train_old.iloc[val])[:,1]])
    xrf_predictions.append([y_train_old[val], best_xrf.predict_proba(X_train_old.iloc[val])[:,1]])

In [165]:
np.mean([roc_auc_score(rf_p[0], rankdata(rf_p[1]) + rankdata(et_p[1]) + rankdata(gb_p[1]) + rankdata(bg_p[1])) for rf_p, et_p, gb_p, bg_p in zip(rfc_predictions, xrf_predictions, gbm_predictions, bg_predictions)])

0.7457898794387926

In [160]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 



In [166]:
best_gbm.fit(X_train_old, y_train_old)
best_xrf.fit(X_train_old, y_train_old)
rfc.fit(X_train_old, y_train_old)
bg.fit(X_train_old, y_train_old)

rfc_predictions_1 = []
gbm_predictions_1 = []
xrf_predictions_1 = []
bg_predictions_1 = []

bg_predictions_1.append(bg.predict_proba(X_train_old)[:,1])
rfc_predictions_1.append(rfc.predict_proba(X_train_old)[:,1])
gbm_predictions_1.append(best_gbm.predict_proba(X_train_old)[:,1])
xrf_predictions_1.append(best_xrf.predict_proba(X_train_old)[:,1])

In [181]:
t = (rankdata(rfc_predictions_1[0]) + rankdata(xrf_predictions_1[0]) + rankdata(gbm_predictions_1[0]) + rankdata(bg_predictions_1[0]))

tr = Find_Optimal_Cutoff(y_train_old, t)[0]
print(tr)

82073.0


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [184]:
coef = tr/(max(t) - min(t))


bg_predictions_1.append(bg.predict_proba(X_test_old)[:,1])
rfc_predictions_1.append(rfc.predict_proba(X_test_old)[:,1])
gbm_predictions_1.append(best_gbm.predict_proba(X_test_old)[:,1])
xrf_predictions_1.append(best_xrf.predict_proba(X_test_old)[:,1])

t_1 = (rankdata(rfc_predictions_1[0]) + rankdata(xrf_predictions_1[0]) + rankdata(gbm_predictions_1[0]) + rankdata(bg_predictions_1[0]))

new_thr = coef * (max(t_1) - min(t_1)) + min(t_1)

In [194]:
t_1 = np.where(t_1 < new_thr , 0, t_1)
t_1 = np.where(t_1 >= new_thr , 1, t_1)
print(np.sum(t_1)/len(t_1))

0.30196831493038884


In [211]:
test['0'] = t_1
header = ['_VAL_']
test['0'].to_csv('ensemble_vote2.csv',sep =',', header = header, index_label = ['_ID_']) 

дальше я тренирую новые модели(два Dart XGB, и два катбуста)

# Dart XGB

In [236]:
num_round_1 = 150

params_1 = {'booster': 'dart',
         'max_depth': 10, 'learning_rate': 0.05,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'weighted',
         'normalize_type': 'tree',
         'rate_drop': 0.25,
         'subsample':0.7, 'colsample_bytree':0.7, 'eval_metric':'auc'}

xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = True)

preds_1 = xgb_1.predict(dtest)

In [245]:
num_round_2 = 150

params_2 = {'booster': 'dart',
         'max_depth': 7, 'learning_rate': 0.08,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'weighted',
         'normalize_type': 'tree',
         'rate_drop': 0.25,
         'subsample':0.6, 'colsample_bytree':0.6, 'eval_metric':'auc'}

xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = True)

preds_2 = xgb_2.predict(dtest)

In [246]:
print(np.corrcoef(preds_1, preds_2)[0][1])
print('preds_1')
print(np.corrcoef(preds_1, y_gbm)[0][1])
print(np.corrcoef(preds_1, y_rfc)[0][1])
print(np.corrcoef(preds_1, y_xrf)[0][1])
print(np.corrcoef(preds_1, y_bg)[0][1])
print('preds_2')
print(np.corrcoef(preds_2, y_gbm)[0][1])
print(np.corrcoef(preds_2, y_rfc)[0][1])
print(np.corrcoef(preds_2, y_xrf)[0][1])
print(np.corrcoef(preds_2, y_bg)[0][1])




0.9734425194676225
preds_1
0.7316078456403736
0.5924411740445731
0.7652934700062765
0.6695967345036121
preds_2
0.7484585306956676
0.564183240383969
0.7715719238095538
0.6859181318761481


# Catboost

In [421]:

category_cols = [i for i in X_train_old.columns if len(X_train_old[i].value_counts()) == 2 ]

cat_dims = [X_train_old.columns.get_loc(i) for i in category_cols[:-1]] 


# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params, train_set, train_label, cat_dims, n_splits=4, cross_val = skf):
    #kf = KFold(n_splits=n_splits,shuffle=True)
    kf = skf
    res = []
    for train_index, test_index in kf.split(train_set, train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims, verbose = True)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    
    print('res = ', np.mean(res))
    return np.mean(res)


# this function runs grid search on several parameters
def catboost_param_tune(params, train_set, train_label, cat_dims=None, n_splits=4, cross_val = skf):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(#ps.grid_search(['border_count']),
                      #ps.grid_search(['ctr_border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms, train_set, train_label, cat_dims, n_splits, skf)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        #print(res,prms,s,'best:',ps.bestscore(),ps.bestparam())
        print('best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()




In [308]:
params = {'depth':[5, 7],
          'iterations':[150],
          'learning_rate':[0.05], 
          'l2_leaf_reg':[1, 3],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

bestparams = catboost_param_tune(params, X_train_old, y_train_old, cat_dims, 4, skf)


Borders generated
0:	learn 0.5	total: 1.61s	remaining: 3m 59s
1:	learn 0.561525406	total: 3.01s	remaining: 3m 42s
2:	learn 0.6422633313	total: 4.35s	remaining: 3m 33s
3:	learn 0.6643448601	total: 5.69s	remaining: 3m 27s
4:	learn 0.6760772193	total: 7.15s	remaining: 3m 27s
5:	learn 0.6733355368	total: 7.78s	remaining: 3m 6s
6:	learn 0.6743682992	total: 9.11s	remaining: 3m 6s
7:	learn 0.6763049803	total: 10.5s	remaining: 3m 7s
8:	learn 0.6800233903	total: 11.7s	remaining: 3m 3s
9:	learn 0.6796083387	total: 13.2s	remaining: 3m 4s
10:	learn 0.6802830256	total: 14.5s	remaining: 3m 3s
11:	learn 0.6779180293	total: 15.2s	remaining: 2m 54s
12:	learn 0.6795903162	total: 16.2s	remaining: 2m 50s
13:	learn 0.681061951	total: 17.5s	remaining: 2m 50s
14:	learn 0.6838695326	total: 19s	remaining: 2m 50s
15:	learn 0.6844824714	total: 20.3s	remaining: 2m 49s
16:	learn 0.6844627886	total: 21.6s	remaining: 2m 48s
17:	learn 0.6842614696	total: 22.3s	remaining: 2m 43s
18:	learn 0.6863693539	total: 23.7s	rem

1:	learn 0.6311592737	total: 4.7s	remaining: 5m 47s
2:	learn 0.6414455402	total: 5.08s	remaining: 4m 8s
3:	learn 0.6414455402	total: 5.19s	remaining: 3m 9s
4:	learn 0.6520887389	total: 6.56s	remaining: 3m 10s
5:	learn 0.6610017859	total: 7.69s	remaining: 3m 4s
6:	learn 0.6645699827	total: 9.15s	remaining: 3m 6s
7:	learn 0.6703895548	total: 10.5s	remaining: 3m 6s
8:	learn 0.6736213772	total: 11.9s	remaining: 3m 6s
9:	learn 0.6817815363	total: 13.3s	remaining: 3m 5s
10:	learn 0.6814328861	total: 14.6s	remaining: 3m 4s
11:	learn 0.6827077623	total: 16.1s	remaining: 3m 5s
12:	learn 0.6838208506	total: 17.6s	remaining: 3m 5s
13:	learn 0.6845490561	total: 19.1s	remaining: 3m 5s
14:	learn 0.6866748694	total: 20.9s	remaining: 3m 8s
15:	learn 0.6853813443	total: 22.7s	remaining: 3m 10s
16:	learn 0.6860754876	total: 24.3s	remaining: 3m 9s
17:	learn 0.6862802341	total: 26s	remaining: 3m 10s
18:	learn 0.6871673906	total: 27.1s	remaining: 3m 7s
19:	learn 0.688614644	total: 28.1s	remaining: 3m 2s
20

4:	learn 0.6558310648	total: 6.2s	remaining: 2m 59s
5:	learn 0.6595628144	total: 7.84s	remaining: 3m 8s
6:	learn 0.6618186541	total: 8.18s	remaining: 2m 47s
7:	learn 0.6638221346	total: 9.81s	remaining: 2m 54s
8:	learn 0.6739415866	total: 11.3s	remaining: 2m 56s
9:	learn 0.6799382554	total: 12.8s	remaining: 2m 59s
10:	learn 0.6801537873	total: 13.5s	remaining: 2m 50s
11:	learn 0.6801537873	total: 13.6s	remaining: 2m 36s
12:	learn 0.6838990351	total: 14.7s	remaining: 2m 35s
13:	learn 0.6838990351	total: 14.9s	remaining: 2m 24s
14:	learn 0.6834015722	total: 16.5s	remaining: 2m 28s
15:	learn 0.6881001467	total: 18s	remaining: 2m 30s
16:	learn 0.688659897	total: 19s	remaining: 2m 28s
17:	learn 0.6904445249	total: 20.6s	remaining: 2m 31s
18:	learn 0.690683288	total: 22.2s	remaining: 2m 33s
19:	learn 0.6928320252	total: 23.8s	remaining: 2m 34s
20:	learn 0.6935330646	total: 25.1s	remaining: 2m 34s
21:	learn 0.6940428948	total: 26.6s	remaining: 2m 34s
22:	learn 0.6956923062	total: 28.3s	remain

6:	learn 0.6505704008	total: 7.33s	remaining: 2m 29s
7:	learn 0.6514118229	total: 7.95s	remaining: 2m 21s
8:	learn 0.6544505908	total: 9.05s	remaining: 2m 21s
9:	learn 0.6549508656	total: 10s	remaining: 2m 20s
10:	learn 0.654209403	total: 10.3s	remaining: 2m 10s
11:	learn 0.6555153077	total: 11.6s	remaining: 2m 13s
12:	learn 0.6670844696	total: 13s	remaining: 2m 16s
13:	learn 0.6651977984	total: 13.3s	remaining: 2m 8s
14:	learn 0.6675267854	total: 14.7s	remaining: 2m 12s
15:	learn 0.6692569889	total: 16.1s	remaining: 2m 14s
16:	learn 0.6730033026	total: 17.4s	remaining: 2m 16s
17:	learn 0.6762352782	total: 18.7s	remaining: 2m 16s
18:	learn 0.6777457876	total: 20s	remaining: 2m 18s
19:	learn 0.6779549799	total: 21.6s	remaining: 2m 20s
20:	learn 0.6779549799	total: 21.7s	remaining: 2m 13s
21:	learn 0.6777115572	total: 22.9s	remaining: 2m 13s
22:	learn 0.678964293	total: 23.9s	remaining: 2m 12s
23:	learn 0.6792058271	total: 25.4s	remaining: 2m 13s
24:	learn 0.6808778036	total: 26.8s	remai

4:	learn 0.670270464	total: 8.75s	remaining: 4m 13s
5:	learn 0.6704564669	total: 9.36s	remaining: 3m 44s
6:	learn 0.6653055001	total: 10.3s	remaining: 3m 30s
7:	learn 0.6669462854	total: 11.7s	remaining: 3m 28s
8:	learn 0.6750617537	total: 14.5s	remaining: 3m 46s
9:	learn 0.6802875884	total: 17.1s	remaining: 3m 58s
10:	learn 0.6802474383	total: 19.7s	remaining: 4m 8s
11:	learn 0.6820055687	total: 22s	remaining: 4m 13s
12:	learn 0.6828209312	total: 23.4s	remaining: 4m 6s
13:	learn 0.6828990022	total: 25.9s	remaining: 4m 11s
14:	learn 0.6835907964	total: 27.5s	remaining: 4m 7s
15:	learn 0.6842926052	total: 30.2s	remaining: 4m 13s
16:	learn 0.6842926052	total: 30.3s	remaining: 3m 57s
17:	learn 0.6847989456	total: 30.7s	remaining: 3m 44s
18:	learn 0.6847989456	total: 30.8s	remaining: 3m 32s
19:	learn 0.6847989456	total: 30.9s	remaining: 3m 20s
20:	learn 0.6864032023	total: 33.5s	remaining: 3m 25s
21:	learn 0.6886869376	total: 36s	remaining: 3m 29s
22:	learn 0.6900141576	total: 37.4s	remain

5:	learn 0.6745837637	total: 13.9s	remaining: 5m 33s
6:	learn 0.6716770336	total: 15.2s	remaining: 5m 10s
7:	learn 0.6741298674	total: 16.1s	remaining: 4m 46s
8:	learn 0.68025485	total: 18.8s	remaining: 4m 55s
9:	learn 0.6833333821	total: 20.8s	remaining: 4m 51s
10:	learn 0.6842629025	total: 23.5s	remaining: 4m 56s
11:	learn 0.6875645687	total: 24.5s	remaining: 4m 42s
12:	learn 0.6881433341	total: 25.2s	remaining: 4m 25s
13:	learn 0.6881433341	total: 25.3s	remaining: 4m 5s
14:	learn 0.6890639385	total: 27.9s	remaining: 4m 11s
15:	learn 0.6890639385	total: 28s	remaining: 3m 54s
16:	learn 0.6895209006	total: 29.8s	remaining: 3m 52s
17:	learn 0.6901189424	total: 32.4s	remaining: 3m 57s
18:	learn 0.6931827214	total: 35s	remaining: 4m 1s
19:	learn 0.6956568385	total: 37.9s	remaining: 4m 6s
20:	learn 0.6968471635	total: 40.4s	remaining: 4m 8s
21:	learn 0.6978138576	total: 42.3s	remaining: 4m 5s
22:	learn 0.6984205278	total: 44.9s	remaining: 4m 8s
23:	learn 0.6978277741	total: 45.9s	remaining

6:	learn 0.6643028245	total: 11.8s	remaining: 4m
7:	learn 0.6721935311	total: 14.4s	remaining: 4m 15s
8:	learn 0.6733273535	total: 15.1s	remaining: 3m 55s
9:	learn 0.6768068746	total: 17.6s	remaining: 4m 6s
10:	learn 0.6793976573	total: 20.1s	remaining: 4m 14s
11:	learn 0.6841965754	total: 22.7s	remaining: 4m 21s
12:	learn 0.6843041486	total: 25.5s	remaining: 4m 29s
13:	learn 0.6862338372	total: 26.9s	remaining: 4m 20s
14:	learn 0.6864324393	total: 29.3s	remaining: 4m 24s
15:	learn 0.6870450121	total: 31.8s	remaining: 4m 26s
16:	learn 0.6893311098	total: 34.3s	remaining: 4m 28s
17:	learn 0.690802614	total: 36.3s	remaining: 4m 26s
18:	learn 0.6922469586	total: 39.1s	remaining: 4m 29s
19:	learn 0.6922469586	total: 39.2s	remaining: 4m 14s
20:	learn 0.6922469586	total: 39.3s	remaining: 4m 1s
21:	learn 0.6921541255	total: 42s	remaining: 4m 4s
22:	learn 0.694632511	total: 44.6s	remaining: 4m 6s
23:	learn 0.6967181894	total: 46.3s	remaining: 4m 3s
24:	learn 0.6976598483	total: 47.4s	remaining

8:	learn 0.6731548953	total: 18.2s	remaining: 4m 44s
9:	learn 0.6726460596	total: 19.2s	remaining: 4m 28s
10:	learn 0.6760062521	total: 21.5s	remaining: 4m 31s
11:	learn 0.6768736901	total: 24.1s	remaining: 4m 37s
12:	learn 0.6768736901	total: 24.2s	remaining: 4m 15s
13:	learn 0.6767931788	total: 25.2s	remaining: 4m 4s
14:	learn 0.6793968455	total: 27.8s	remaining: 4m 10s
15:	learn 0.6830799526	total: 30.4s	remaining: 4m 14s
16:	learn 0.6840603614	total: 33.2s	remaining: 4m 19s
17:	learn 0.6850836986	total: 34.2s	remaining: 4m 10s
18:	learn 0.6835713398	total: 35.2s	remaining: 4m 2s
19:	learn 0.6865584419	total: 37.7s	remaining: 4m 4s
20:	learn 0.6875877064	total: 39.4s	remaining: 4m 2s
21:	learn 0.6875877064	total: 39.6s	remaining: 3m 50s
22:	learn 0.6869518693	total: 40.2s	remaining: 3m 41s
23:	learn 0.6872296334	total: 42.9s	remaining: 3m 45s
24:	learn 0.6872296334	total: 43.1s	remaining: 3m 35s
25:	learn 0.6891201016	total: 45.6s	remaining: 3m 37s
26:	learn 0.6903631656	total: 47s	

In [309]:
clf_cb1 = cb.CatBoostClassifier(**bestparams)
clf_cb1.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
res_cb1 = clf_cb1.predict_proba(X_test_old)[:,1]

In [326]:
print(np.corrcoef(res_cb1, y_gbm)[0][1])


0.7545372242555904


# Catboost2

In [327]:
params2 = {'depth':[3],
          'iterations':[200, 250],
          'learning_rate':[0.1, 0.05], 
          'l2_leaf_reg':[5, 7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

In [329]:
bestparams2 = catboost_param_tune(params2, X_train_old, y_train_old, cat_dims, 4, skf)



Borders generated
0:	learn 0.6039888522	total: 2.3s	remaining: 7m 38s
1:	learn 0.6039888522	total: 2.41s	remaining: 3m 59s
2:	learn 0.6364303693	total: 3.08s	remaining: 3m 22s
3:	learn 0.6354674152	total: 3.81s	remaining: 3m 6s
4:	learn 0.6497279399	total: 4.47s	remaining: 2m 54s
5:	learn 0.6696182316	total: 5.12s	remaining: 2m 45s
6:	learn 0.6684829761	total: 5.74s	remaining: 2m 38s
7:	learn 0.6723510303	total: 6.54s	remaining: 2m 36s
8:	learn 0.6753046021	total: 7.23s	remaining: 2m 33s
9:	learn 0.6797992508	total: 7.87s	remaining: 2m 29s
10:	learn 0.679421231	total: 8.23s	remaining: 2m 21s
11:	learn 0.6841039156	total: 8.92s	remaining: 2m 19s
12:	learn 0.6833496411	total: 9.6s	remaining: 2m 18s
13:	learn 0.6862351173	total: 10.3s	remaining: 2m 17s
14:	learn 0.6871309498	total: 11.1s	remaining: 2m 16s
15:	learn 0.6871122671	total: 11.7s	remaining: 2m 14s
16:	learn 0.6912094356	total: 12.3s	remaining: 2m 12s
17:	learn 0.6933084034	total: 12.9s	remaining: 2m 10s
18:	learn 0.6945953689	t

152:	learn 0.7435114815	total: 1m 43s	remaining: 31.7s
153:	learn 0.7435552923	total: 1m 43s	remaining: 31s
154:	learn 0.7437919174	total: 1m 44s	remaining: 30.4s
155:	learn 0.7438175685	total: 1m 45s	remaining: 29.7s
156:	learn 0.7440821718	total: 1m 45s	remaining: 29s
157:	learn 0.7441866064	total: 1m 46s	remaining: 28.3s
158:	learn 0.7443618102	total: 1m 46s	remaining: 27.6s
159:	learn 0.7445152982	total: 1m 47s	remaining: 26.9s
160:	learn 0.7445649922	total: 1m 48s	remaining: 26.2s
161:	learn 0.7446688385	total: 1m 48s	remaining: 25.5s
162:	learn 0.744815515	total: 1m 49s	remaining: 24.8s
163:	learn 0.7448717329	total: 1m 50s	remaining: 24.1s
164:	learn 0.7449219237	total: 1m 50s	remaining: 23.5s
165:	learn 0.7450142322	total: 1m 51s	remaining: 22.8s
166:	learn 0.7450784252	total: 1m 51s	remaining: 22.1s
167:	learn 0.7452272066	total: 1m 52s	remaining: 21.4s
168:	learn 0.7455272207	total: 1m 53s	remaining: 20.8s
169:	learn 0.7458016034	total: 1m 53s	remaining: 20.1s
170:	learn 0.74

105:	learn 0.7315065238	total: 1m 15s	remaining: 1m 6s
106:	learn 0.7315864797	total: 1m 15s	remaining: 1m 5s
107:	learn 0.7317488755	total: 1m 16s	remaining: 1m 5s
108:	learn 0.7321751873	total: 1m 17s	remaining: 1m 4s
109:	learn 0.7325536445	total: 1m 17s	remaining: 1m 3s
110:	learn 0.7327353494	total: 1m 18s	remaining: 1m 2s
111:	learn 0.7332875434	total: 1m 19s	remaining: 1m 2s
112:	learn 0.7333457128	total: 1m 19s	remaining: 1m 1s
113:	learn 0.7335523027	total: 1m 20s	remaining: 1m
114:	learn 0.7335931696	total: 1m 21s	remaining: 1m
115:	learn 0.7343308141	total: 1m 21s	remaining: 59.2s
116:	learn 0.7346458269	total: 1m 22s	remaining: 58.4s
117:	learn 0.7348267213	total: 1m 23s	remaining: 57.7s
118:	learn 0.7350157734	total: 1m 23s	remaining: 57s
119:	learn 0.7351749009	total: 1m 24s	remaining: 56.4s
120:	learn 0.7353757058	total: 1m 25s	remaining: 55.7s
121:	learn 0.7354845797	total: 1m 26s	remaining: 55s
122:	learn 0.7356980135	total: 1m 26s	remaining: 54.4s
123:	learn 0.7360103

57:	learn 0.722569507	total: 37.9s	remaining: 1m 32s
58:	learn 0.7230878413	total: 38.5s	remaining: 1m 32s
59:	learn 0.7234989424	total: 39.2s	remaining: 1m 31s
60:	learn 0.7240113742	total: 39.8s	remaining: 1m 30s
61:	learn 0.7244593287	total: 40.4s	remaining: 1m 30s
62:	learn 0.7247775379	total: 41s	remaining: 1m 29s
63:	learn 0.7251297441	total: 41.6s	remaining: 1m 28s
64:	learn 0.7253962224	total: 42.3s	remaining: 1m 27s
65:	learn 0.7254169762	total: 42.9s	remaining: 1m 27s
66:	learn 0.7255828625	total: 43.6s	remaining: 1m 26s
67:	learn 0.7261187215	total: 44.2s	remaining: 1m 25s
68:	learn 0.7265926199	total: 44.9s	remaining: 1m 25s
69:	learn 0.7269709856	total: 45.5s	remaining: 1m 24s
70:	learn 0.7273289244	total: 46s	remaining: 1m 23s
71:	learn 0.7275130217	total: 46.7s	remaining: 1m 22s
72:	learn 0.7276949293	total: 47.3s	remaining: 1m 22s
73:	learn 0.7278198637	total: 47.9s	remaining: 1m 21s
74:	learn 0.7280629475	total: 48.6s	remaining: 1m 20s
75:	learn 0.7283032337	total: 49.

9:	learn 0.6704460084	total: 7.56s	remaining: 2m 23s
10:	learn 0.6728695505	total: 8.19s	remaining: 2m 20s
11:	learn 0.6734310454	total: 8.8s	remaining: 2m 17s
12:	learn 0.6738124091	total: 9.43s	remaining: 2m 15s
13:	learn 0.6758296434	total: 10.1s	remaining: 2m 13s
14:	learn 0.6755722946	total: 10.7s	remaining: 2m 12s
15:	learn 0.6799176207	total: 11.3s	remaining: 2m 10s
16:	learn 0.6848252715	total: 12s	remaining: 2m 8s
17:	learn 0.6882992059	total: 12.6s	remaining: 2m 7s
18:	learn 0.6908721058	total: 13.2s	remaining: 2m 5s
19:	learn 0.691254489	total: 13.8s	remaining: 2m 4s
20:	learn 0.6942432379	total: 14.5s	remaining: 2m 3s
21:	learn 0.6960301849	total: 15.1s	remaining: 2m 2s
22:	learn 0.6971854118	total: 15.8s	remaining: 2m 1s
23:	learn 0.6984209017	total: 16.4s	remaining: 2m
24:	learn 0.6982263282	total: 17s	remaining: 1m 59s
25:	learn 0.6992092595	total: 17.7s	remaining: 1m 58s
26:	learn 0.7019190222	total: 18.2s	remaining: 1m 56s
27:	learn 0.7028672526	total: 18.8s	remaining:

...

122:	learn 0.7226492082	total: 1m 21s	remaining: 1m 23s
123:	learn 0.7227676384	total: 1m 21s	remaining: 1m 23s
124:	learn 0.7229945954	total: 1m 22s	remaining: 1m 22s
125:	learn 0.7231798465	total: 1m 23s	remaining: 1m 21s
126:	learn 0.7234201912	total: 1m 23s	remaining: 1m 21s
127:	learn 0.7235222267	total: 1m 24s	remaining: 1m 20s
128:	learn 0.7236232948	total: 1m 25s	remaining: 1m 19s
129:	learn 0.7237218265	total: 1m 25s	remaining: 1m 19s
130:	learn 0.7240502045	total: 1m 26s	remaining: 1m 18s
131:	learn 0.7242429404	total: 1m 26s	remaining: 1m 17s
132:	learn 0.7243135527	total: 1m 27s	remaining: 1m 16s
133:	learn 0.7243643973	total: 1m 28s	remaining: 1m 16s
134:	learn 0.7246517624	total: 1m 28s	remaining: 1m 15s
135:	learn 0.7247950265	total: 1m 29s	remaining: 1m 14s
136:	learn 0.7250878042	total: 1m 29s	remaining: 1m 14s
137:	learn 0.7251880159	total: 1m 30s	remaining: 1m 13s
138:	learn 0.7253067926	total: 1m 30s	remaining: 1m 12s
139:	learn 0.7253457268	total: 1m 31s	remaining:

173:	learn 0.7280716151	total: 1m 57s	remaining: 51.5s
174:	learn 0.7281652064	total: 1m 58s	remaining: 50.8s
175:	learn 0.7282691387	total: 1m 59s	remaining: 50.1s
176:	learn 0.7283132739	total: 2m	remaining: 49.5s
177:	learn 0.7285112812	total: 2m	remaining: 48.9s
178:	learn 0.7287993631	total: 2m 1s	remaining: 48.2s
179:	learn 0.7288687297	total: 2m 2s	remaining: 47.6s
180:	learn 0.7290043777	total: 2m 3s	remaining: 46.9s
181:	learn 0.7291043749	total: 2m 3s	remaining: 46.2s
182:	learn 0.7291230696	total: 2m 4s	remaining: 45.6s
183:	learn 0.7292134841	total: 2m 5s	remaining: 45s
184:	learn 0.7293171157	total: 2m 6s	remaining: 44.3s
185:	learn 0.729416956	total: 2m 6s	remaining: 43.7s
186:	learn 0.7294923886	total: 2m 7s	remaining: 43s
187:	learn 0.7296471888	total: 2m 8s	remaining: 42.3s
188:	learn 0.729876598	total: 2m 9s	remaining: 41.7s
189:	learn 0.7301519466	total: 2m 9s	remaining: 41s
190:	learn 0.7302141752	total: 2m 10s	remaining: 40.4s
191:	learn 0.7303402143	total: 2m 11s	

75:	learn 0.7101836521	total: 51.6s	remaining: 1m 58s
76:	learn 0.7104585235	total: 52.3s	remaining: 1m 57s
77:	learn 0.7107239887	total: 52.9s	remaining: 1m 56s
78:	learn 0.7110278696	total: 53.6s	remaining: 1m 55s
79:	learn 0.7117839735	total: 54.2s	remaining: 1m 55s
80:	learn 0.7117748745	total: 55s	remaining: 1m 54s
81:	learn 0.7119355773	total: 55.7s	remaining: 1m 54s
82:	learn 0.7128239431	total: 56.4s	remaining: 1m 53s
83:	learn 0.7128239431	total: 56.5s	remaining: 1m 51s
84:	learn 0.7128722357	total: 57.1s	remaining: 1m 50s
85:	learn 0.7138438715	total: 57.7s	remaining: 1m 50s
86:	learn 0.7138438715	total: 57.9s	remaining: 1m 48s
87:	learn 0.7140955836	total: 58.5s	remaining: 1m 47s
88:	learn 0.7144802049	total: 59.2s	remaining: 1m 47s
89:	learn 0.7147426567	total: 59.8s	remaining: 1m 46s
90:	learn 0.7153729437	total: 1m	remaining: 1m 45s
91:	learn 0.7156946565	total: 1m 1s	remaining: 1m 44s
92:	learn 0.715791673	total: 1m 1s	remaining: 1m 44s
93:	learn 0.7160776109	total: 1m 2

225:	learn 0.7384994256	total: 2m 32s	remaining: 16.2s
226:	learn 0.7386123523	total: 2m 33s	remaining: 15.5s
227:	learn 0.7387002567	total: 2m 33s	remaining: 14.8s
228:	learn 0.7387704404	total: 2m 34s	remaining: 14.2s
229:	learn 0.738891041	total: 2m 35s	remaining: 13.5s
230:	learn 0.7389220573	total: 2m 36s	remaining: 12.9s
231:	learn 0.7391330463	total: 2m 37s	remaining: 12.2s
232:	learn 0.7392140742	total: 2m 38s	remaining: 11.5s
233:	learn 0.7392616739	total: 2m 38s	remaining: 10.9s
234:	learn 0.7394278413	total: 2m 39s	remaining: 10.2s
235:	learn 0.7396250511	total: 2m 40s	remaining: 9.49s
236:	learn 0.7397432267	total: 2m 40s	remaining: 8.82s
237:	learn 0.739851996	total: 2m 41s	remaining: 8.14s
238:	learn 0.7398832345	total: 2m 42s	remaining: 7.46s
239:	learn 0.7399183231	total: 2m 43s	remaining: 6.79s
240:	learn 0.7399783423	total: 2m 43s	remaining: 6.12s
241:	learn 0.7401428691	total: 2m 44s	remaining: 5.44s
242:	learn 0.7403674673	total: 2m 45s	remaining: 4.76s
243:	learn 0

127:	learn 0.7178704003	total: 1m 31s	remaining: 1m 26s
128:	learn 0.7182217357	total: 1m 31s	remaining: 1m 26s
129:	learn 0.7182701863	total: 1m 32s	remaining: 1m 25s
130:	learn 0.7184870572	total: 1m 33s	remaining: 1m 24s
131:	learn 0.7185172359	total: 1m 34s	remaining: 1m 24s
132:	learn 0.7185814489	total: 1m 34s	remaining: 1m 23s
133:	learn 0.7186427735	total: 1m 35s	remaining: 1m 22s
134:	learn 0.7187605475	total: 1m 36s	remaining: 1m 21s
135:	learn 0.7188039792	total: 1m 37s	remaining: 1m 21s
136:	learn 0.718991802	total: 1m 38s	remaining: 1m 20s
137:	learn 0.7193254667	total: 1m 39s	remaining: 1m 20s
138:	learn 0.7195841356	total: 1m 39s	remaining: 1m 19s
139:	learn 0.719731291	total: 1m 40s	remaining: 1m 19s
140:	learn 0.7199252959	total: 1m 41s	remaining: 1m 18s
141:	learn 0.7201606021	total: 1m 42s	remaining: 1m 17s
142:	learn 0.7203208014	total: 1m 43s	remaining: 1m 17s
143:	learn 0.7205041477	total: 1m 43s	remaining: 1m 16s
144:	learn 0.7205000895	total: 1m 44s	remaining: 1

In [330]:
clf_cb2 = cb.CatBoostClassifier(**bestparams2)
clf_cb2.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
res_cb2 = clf_cb2.predict_proba(X_test_old)[:,1]

In [332]:
print(np.corrcoef(res_cb2, preds_2)[0][1])

print(np.corrcoef(res_cb1, res_cb2)[0][1])



0.9357028235171487
0.9796548285297352


# Ensembling 1
на всех фичах и еще предикты моделей

catboost


In [ ]:
rfc_predictions = []
gbm_predictions = []
xrf_predictions = []
bg_predictions = []
xgb_1_predictions = []
xgb_2_predictions = []
clf_cb1_predictions = []
clf_cb2_predictions = []




for train, val in skf.split(X_train_old, y_train_old):
    
    best_gbm.fit(X_train_old.iloc[train], y_train_old[train])
    best_xrf.fit(X_train_old.iloc[train], y_train_old[train])
    rfc.fit(X_train_old.iloc[train], y_train_old[train])
    bg.fit(X_train_old.iloc[train], y_train_old[train])
    

    dtrain = xgb.DMatrix(data = X_train_old.iloc[train], label = y_train_old.iloc[train])
    dtest = xgb.DMatrix(X_train_old.iloc[val])

    xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = False)
    xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = False)  
    clf_cb1.fit(X_train_old.iloc[train], np.ravel(y_train_old[train]),  cat_features = cat_dims)
    clf_cb2.fit(X_train_old.iloc[train], np.ravel(y_train_old[train]), cat_features = cat_dims)
    
    preds1 = xgb_1.predict(dtest)
    preds2 = xgb_2.predict(dtest)    
    
    bg_predictions.append([y_train_old[val], bg.predict_proba(X_train_old.iloc[val])[:,1]])
    rfc_predictions.append([y_train_old[val], rfc.predict_proba(X_train_old.iloc[val])[:,1]])
    gbm_predictions.append([y_train_old[val], best_gbm.predict_proba(X_train_old.iloc[val])[:,1]])
    xrf_predictions.append([y_train_old[val], best_xrf.predict_proba(X_train_old.iloc[val])[:,1]])
    
    xgb_1_predictions.append([y_train_old[val], preds1])
    xgb_2_predictions.append([y_train_old[val], preds2])
    clf_cb1_predictions.append([y_train_old[val], clf_cb1.predict_proba(X_train_old.iloc[val])[:,1]])
    clf_cb2_predictions.append([y_train_old[val], clf_cb2.predict_proba(X_train_old.iloc[val])[:,1]])


    
    
    

In [363]:
best_gbm.fit(X_train_old, y_train_old)
best_xrf.fit(X_train_old, y_train_old)
rfc.fit(X_train_old, y_train_old)
bg.fit(X_train_old, y_train_old)
    
dtrain = xgb.DMatrix(data = X_train_old, label = y_train_old)
dtest = xgb.DMatrix(X_test_old)

xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = False)
xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = False)  
clf_cb1.fit(X_train_old, np.ravel(y_train_old),  cat_features = cat_dims)
clf_cb2.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
rfc_preds_test = rfc.predict_proba(X_test_old)[:,1]
xrf_preds_test = best_xrf.predict_proba(X_test_old)[:,1]
gbm_preds_test = best_gbm.predict_proba(X_test_old)[:,1]
bg_preds_test = bg.predict_proba(X_test_old)[:,1]

xgb_1_preds_test = xgb_1.predict(dtest)
xgb_2_preds_test = xgb_2.predict(dtest)
clf_cb_1_preds_test = clf_cb1.predict_proba(X_test_old)[:,1]
clf_cb_2_preds_test = clf_cb2.predict_proba(X_test_old)[:,1]


X_test_old["rfc_preds"] = rfc_preds_test
X_test_old["xrf_preds"] = xrf_preds_test
X_test_old["gbm_preds"] = gbm_preds_test
X_test_old["bg_preds"] = bg_preds_test

X_test_old["xgb_1_preds"] = xgb_1_preds_test
X_test_old["xgb_2_preds"] = xgb_2_preds_test
X_test_old["clf_cb_1_preds"] = clf_cb_1_preds_test
X_test_old["clf_cb_2_preds"] = clf_cb_2_preds_test





X_train_old["rfc_preds"] = 0
X_train_old["xrf_preds"] = 0
X_train_old["gbm_preds"] = 0
X_train_old["bg_preds"] = 0

X_train_old["xgb_1_preds"] = 0
X_train_old["xgb_2_preds"] = 0
X_train_old["clf_cb_1_preds"] = 0
X_train_old["clf_cb_2_preds"] = 0



for i, fold in enumerate(skf.split(X_train_old, y_train_old)):
    train, val = fold[0], fold[1]
    
    
    
    X_train_old.iloc[val, -8] = rfc_predictions[i][1]
    X_train_old.iloc[val, -7] = xrf_predictions[i][1]
    X_train_old.iloc[val, -6] = gbm_predictions[i][1]
    X_train_old.iloc[val, -5] = bg_predictions[i][1]
    
    X_train_old.iloc[val, -4] = xgb_1_predictions[i][1]
    X_train_old.iloc[val, -3] = xgb_2_predictions[i][1]
    X_train_old.iloc[val, -2] = clf_cb1_predictions[i][1]
    X_train_old.iloc[val, -1] = clf_cb2_predictions[i][1]



In [365]:
params3 = {'depth':[2, 3],
          'iterations':[200, 300],
          'learning_rate':[0.1], 
          'l2_leaf_reg':[7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

bestparams3 = catboost_param_tune(params3, X_train_old, y_train_old, cat_dims, 4, skf)



Borders generated
0:	learn 0.6375663242	total: 2.47s	remaining: 12m 19s
1:	learn 0.7019900058	total: 2.81s	remaining: 6m 58s
2:	learn 0.7074872749	total: 3.26s	remaining: 5m 22s
3:	learn 0.7098726339	total: 3.59s	remaining: 4m 25s
4:	learn 0.7107992947	total: 3.97s	remaining: 3m 54s
5:	learn 0.7111453029	total: 4.3s	remaining: 3m 30s
6:	learn 0.7115844956	total: 4.63s	remaining: 3m 14s
7:	learn 0.7129107547	total: 4.98s	remaining: 3m 1s
8:	learn 0.7263393327	total: 5.31s	remaining: 2m 51s
9:	learn 0.7316167179	total: 5.64s	remaining: 2m 43s
10:	learn 0.7325143938	total: 5.97s	remaining: 2m 36s
11:	learn 0.7326520624	total: 6.3s	remaining: 2m 31s
12:	learn 0.7341043477	total: 6.63s	remaining: 2m 26s
13:	learn 0.7346657686	total: 6.97s	remaining: 2m 22s
14:	learn 0.7348567657	total: 7.31s	remaining: 2m 18s
15:	learn 0.7405255256	total: 7.63s	remaining: 2m 15s
16:	learn 0.7407390426	total: 7.97s	remaining: 2m 12s
17:	learn 0.7408145315	total: 8.31s	remaining: 2m 10s
18:	learn 0.7410573537

153:	learn 0.7550379328	total: 57.9s	remaining: 54.8s
154:	learn 0.7550629432	total: 58.2s	remaining: 54.5s
155:	learn 0.755156631	total: 58.6s	remaining: 54.1s
156:	learn 0.7551572717	total: 58.9s	remaining: 53.6s
157:	learn 0.75521145	total: 59.3s	remaining: 53.3s
158:	learn 0.7552125548	total: 59.6s	remaining: 52.8s
159:	learn 0.7552617455	total: 59.9s	remaining: 52.4s
160:	learn 0.7553226046	total: 1m	remaining: 52s
161:	learn 0.7553508966	total: 1m	remaining: 51.6s
162:	learn 0.7553775282	total: 1m	remaining: 51.2s
163:	learn 0.7553878436	total: 1m 1s	remaining: 50.8s
164:	learn 0.7553875167	total: 1m 1s	remaining: 50.4s
165:	learn 0.7554167762	total: 1m 1s	remaining: 50s
166:	learn 0.7554215089	total: 1m 2s	remaining: 49.6s
167:	learn 0.7554166062	total: 1m 2s	remaining: 49.2s
168:	learn 0.7554208029	total: 1m 3s	remaining: 48.8s
169:	learn 0.7554287715	total: 1m 3s	remaining: 48.4s
170:	learn 0.7554297913	total: 1m 3s	remaining: 48.1s
171:	learn 0.7555162165	total: 1m 4s	remaini

4:	learn 0.7133905831	total: 4.72s	remaining: 4m 38s
5:	learn 0.7143319086	total: 5.16s	remaining: 4m 12s
6:	learn 0.7167349987	total: 5.5s	remaining: 3m 50s
7:	learn 0.7262216537	total: 5.96s	remaining: 3m 37s
8:	learn 0.7265937115	total: 6.45s	remaining: 3m 28s
9:	learn 0.7270271483	total: 6.94s	remaining: 3m 21s
10:	learn 0.7358095739	total: 7.44s	remaining: 3m 15s
11:	learn 0.7363537541	total: 7.91s	remaining: 3m 9s
12:	learn 0.7365646842	total: 8.41s	remaining: 3m 5s
13:	learn 0.7370304249	total: 8.9s	remaining: 3m 1s
14:	learn 0.737220536	total: 9.39s	remaining: 2m 58s
15:	learn 0.739416631	total: 9.9s	remaining: 2m 55s
16:	learn 0.7394357702	total: 10.4s	remaining: 2m 52s
17:	learn 0.7396569564	total: 10.8s	remaining: 2m 48s
18:	learn 0.7397129948	total: 11.1s	remaining: 2m 44s
19:	learn 0.7397506196	total: 11.4s	remaining: 2m 40s
20:	learn 0.7401597205	total: 11.8s	remaining: 2m 36s
21:	learn 0.7412383215	total: 12.1s	remaining: 2m 32s
22:	learn 0.7418146161	total: 12.4s	remain

157:	learn 0.7494250259	total: 1m 6s	remaining: 59.9s
158:	learn 0.7494484596	total: 1m 6s	remaining: 59.4s
159:	learn 0.7494649973	total: 1m 7s	remaining: 58.9s
160:	learn 0.749551699	total: 1m 7s	remaining: 58.4s
161:	learn 0.7496304784	total: 1m 8s	remaining: 57.9s
162:	learn 0.7496300731	total: 1m 8s	remaining: 57.5s
163:	learn 0.7496949752	total: 1m 8s	remaining: 57s
164:	learn 0.7496966747	total: 1m 9s	remaining: 56.6s
165:	learn 0.7497014726	total: 1m 9s	remaining: 56.2s
166:	learn 0.7497243115	total: 1m 9s	remaining: 55.7s
167:	learn 0.7497854289	total: 1m 10s	remaining: 55.2s
168:	learn 0.7499216521	total: 1m 10s	remaining: 54.8s
169:	learn 0.7499319276	total: 1m 11s	remaining: 54.4s
170:	learn 0.7499487921	total: 1m 11s	remaining: 53.9s
171:	learn 0.7499490274	total: 1m 11s	remaining: 53.5s
172:	learn 0.7499518774	total: 1m 12s	remaining: 53.1s
173:	learn 0.7499539822	total: 1m 12s	remaining: 52.6s
174:	learn 0.7499979344	total: 1m 13s	remaining: 52.2s
175:	learn 0.7500404355

8:	learn 0.7336550191	total: 5.94s	remaining: 3m 11s
9:	learn 0.7342257968	total: 6.41s	remaining: 3m 5s
10:	learn 0.7355059153	total: 6.78s	remaining: 2m 58s
11:	learn 0.7362340163	total: 7.13s	remaining: 2m 51s
12:	learn 0.7377101026	total: 7.49s	remaining: 2m 45s
13:	learn 0.7450315873	total: 7.91s	remaining: 2m 41s
14:	learn 0.7452924507	total: 8.38s	remaining: 2m 39s
15:	learn 0.7452902936	total: 8.77s	remaining: 2m 35s
16:	learn 0.7455407834	total: 9.15s	remaining: 2m 32s
17:	learn 0.7456537493	total: 9.55s	remaining: 2m 29s
18:	learn 0.7464798015	total: 9.98s	remaining: 2m 27s
19:	learn 0.7468615009	total: 10.4s	remaining: 2m 25s
20:	learn 0.7472362518	total: 10.8s	remaining: 2m 23s
21:	learn 0.7472362518	total: 10.9s	remaining: 2m 18s
22:	learn 0.7479131057	total: 11.3s	remaining: 2m 16s
23:	learn 0.7480488909	total: 11.7s	remaining: 2m 14s
24:	learn 0.7489126856	total: 12.1s	remaining: 2m 12s
25:	learn 0.7495803686	total: 12.5s	remaining: 2m 11s
26:	learn 0.7500470114	total: 1

...

164:	learn 0.7532603809	total: 1m 48s	remaining: 1m 28s
165:	learn 0.7532604593	total: 1m 48s	remaining: 1m 27s
166:	learn 0.7532679503	total: 1m 49s	remaining: 1m 27s
167:	learn 0.7532899003	total: 1m 50s	remaining: 1m 26s
168:	learn 0.753294306	total: 1m 51s	remaining: 1m 26s
169:	learn 0.7533113143	total: 1m 52s	remaining: 1m 25s
170:	learn 0.7533449255	total: 1m 53s	remaining: 1m 25s
171:	learn 0.7534963921	total: 1m 54s	remaining: 1m 24s
172:	learn 0.7536057628	total: 1m 54s	remaining: 1m 24s
173:	learn 0.7536845029	total: 1m 55s	remaining: 1m 23s
174:	learn 0.753709865	total: 1m 56s	remaining: 1m 23s
175:	learn 0.7538660641	total: 1m 57s	remaining: 1m 22s
176:	learn 0.7539525828	total: 1m 57s	remaining: 1m 21s
177:	learn 0.7540102358	total: 1m 59s	remaining: 1m 21s
178:	learn 0.7540274663	total: 2m	remaining: 1m 21s
179:	learn 0.7540884399	total: 2m 1s	remaining: 1m 20s
180:	learn 0.7540903486	total: 2m 2s	remaining: 1m 20s
181:	learn 0.7541971569	total: 2m 4s	remaining: 1m 20s
1

14:	learn 0.7446235584	total: 10.9s	remaining: 3m 27s
15:	learn 0.7446601308	total: 11.6s	remaining: 3m 25s
16:	learn 0.7455568766	total: 12.1s	remaining: 3m 22s
17:	learn 0.7460120802	total: 12.8s	remaining: 3m 20s
18:	learn 0.7460617259	total: 13.5s	remaining: 3m 20s
19:	learn 0.7467347624	total: 14.2s	remaining: 3m 18s
20:	learn 0.7467347166	total: 14.9s	remaining: 3m 17s
21:	learn 0.7467643406	total: 15.3s	remaining: 3m 13s
22:	learn 0.7473791358	total: 15.8s	remaining: 3m 10s
23:	learn 0.7475240854	total: 16.4s	remaining: 3m 9s
24:	learn 0.747404426	total: 17.1s	remaining: 3m 8s
25:	learn 0.7475065541	total: 17.5s	remaining: 3m 4s
26:	learn 0.7481234215	total: 18.1s	remaining: 3m 2s
27:	learn 0.748943192	total: 18.6s	remaining: 3m 1s
28:	learn 0.749130525	total: 19.5s	remaining: 3m 2s
29:	learn 0.7491519978	total: 20.4s	remaining: 3m 3s
30:	learn 0.7494912353	total: 21.2s	remaining: 3m 3s
31:	learn 0.7498805629	total: 21.8s	remaining: 3m 2s
32:	learn 0.749965199	total: 22.7s	remai

165:	learn 0.7588536291	total: 1m 52s	remaining: 1m 30s
166:	learn 0.7589117789	total: 1m 53s	remaining: 1m 30s
167:	learn 0.7589229826	total: 1m 53s	remaining: 1m 29s
168:	learn 0.7589793152	total: 1m 54s	remaining: 1m 28s
169:	learn 0.7589805572	total: 1m 55s	remaining: 1m 28s
170:	learn 0.7590721614	total: 1m 56s	remaining: 1m 27s
171:	learn 0.7591340893	total: 1m 56s	remaining: 1m 26s
172:	learn 0.7591629158	total: 1m 57s	remaining: 1m 26s
173:	learn 0.7592894124	total: 1m 57s	remaining: 1m 25s
174:	learn 0.7593522294	total: 1m 58s	remaining: 1m 24s
175:	learn 0.7595241294	total: 1m 59s	remaining: 1m 23s
176:	learn 0.7596294605	total: 1m 59s	remaining: 1m 23s
177:	learn 0.7597291178	total: 2m	remaining: 1m 22s
178:	learn 0.7597623499	total: 2m 1s	remaining: 1m 22s
179:	learn 0.7597994518	total: 2m 2s	remaining: 1m 21s
180:	learn 0.759801099	total: 2m 3s	remaining: 1m 21s
181:	learn 0.7598463455	total: 2m 4s	remaining: 1m 20s
182:	learn 0.7598889904	total: 2m 5s	remaining: 1m 20s
18

15:	learn 0.7421416209	total: 11.5s	remaining: 3m 23s
16:	learn 0.7425570648	total: 12.2s	remaining: 3m 23s
17:	learn 0.7447122989	total: 13.1s	remaining: 3m 24s
18:	learn 0.7448054817	total: 14.3s	remaining: 3m 31s
19:	learn 0.744893822	total: 15.1s	remaining: 3m 32s
20:	learn 0.7452011702	total: 15.8s	remaining: 3m 30s
21:	learn 0.7455092372	total: 16.5s	remaining: 3m 28s
22:	learn 0.745861958	total: 17.1s	remaining: 3m 25s
23:	learn 0.7459042657	total: 17.5s	remaining: 3m 20s
24:	learn 0.7458859611	total: 18.1s	remaining: 3m 19s
25:	learn 0.7461080078	total: 19s	remaining: 3m 20s
26:	learn 0.7463619584	total: 19.6s	remaining: 3m 18s
27:	learn 0.7464943904	total: 20.3s	remaining: 3m 17s
28:	learn 0.7465928665	total: 20.6s	remaining: 3m 12s
29:	learn 0.7466992108	total: 21.4s	remaining: 3m 12s
30:	learn 0.747094762	total: 22.2s	remaining: 3m 12s
31:	learn 0.7473275588	total: 23.3s	remaining: 3m 14s
32:	learn 0.7475712167	total: 24.1s	remaining: 3m 15s
33:	learn 0.7475454296	total: 25.

166:	learn 0.7587237324	total: 2m 7s	remaining: 1m 41s
167:	learn 0.7587624589	total: 2m 8s	remaining: 1m 40s
168:	learn 0.7587694905	total: 2m 8s	remaining: 1m 39s
169:	learn 0.7588970671	total: 2m 9s	remaining: 1m 39s
170:	learn 0.7589402504	total: 2m 10s	remaining: 1m 38s
171:	learn 0.7590382887	total: 2m 11s	remaining: 1m 37s
172:	learn 0.7590459739	total: 2m 12s	remaining: 1m 37s
173:	learn 0.7591547818	total: 2m 13s	remaining: 1m 36s
174:	learn 0.759188973	total: 2m 14s	remaining: 1m 35s
175:	learn 0.7591983312	total: 2m 14s	remaining: 1m 34s
176:	learn 0.7594349901	total: 2m 15s	remaining: 1m 34s
177:	learn 0.759432912	total: 2m 15s	remaining: 1m 33s
178:	learn 0.7594625156	total: 2m 16s	remaining: 1m 32s
179:	learn 0.7595233173	total: 2m 17s	remaining: 1m 31s
180:	learn 0.75956637	total: 2m 18s	remaining: 1m 30s
181:	learn 0.7596333539	total: 2m 18s	remaining: 1m 29s
182:	learn 0.7596816607	total: 2m 19s	remaining: 1m 29s
183:	learn 0.7597509188	total: 2m 19s	remaining: 1m 28s


In [366]:
clf_cb3 = cb.CatBoostClassifier(**bestparams3)
clf_cb3.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)

res_cb3 = clf_cb3.predict_proba(X_test_old)[:,1]

yt = clf_cb3.predict_proba(X_train_old)[:,1]
tr = Find_Optimal_Cutoff(y_train_old, yt)[0]


res_cb3 = np.where(res_cb3 < tr , 0, res_cb3)
res_cb3 = np.where(res_cb3 >= tr , 1, res_cb3)

# Ensembling 2 
на всех фичах и еще предикты моделей

extra trees

он и оказался лучшим на лидерборде среди всех трех ансамблей, CV - 0.7467, LB - 0.6868

In [382]:
parameters = {'max_features': [5, 15, 25], 'min_samples_leaf': [1, 3, 5], 'max_depth': [5,15,20], 'class_weight':['balanced']}

etc = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=2, scoring='roc_auc')
etcv.fit(X_train_old, y_train_old)

Fitting 4 folds for each of 27 candidates, totalling 108 fits
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1, total=   2.6s
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1, total=   3.2s
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1, total=   4.0s
[CV] max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=5, max_features=5, class_weight=balanced, min_samples_leaf=1, total=   4.1s
[CV] max_depth=5, max_features=

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min


[CV]  max_depth=5, max_features=25, class_weight=balanced, min_samples_leaf=5, total=  11.2s
[CV] max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=5, max_features=25, class_weight=balanced, min_samples_leaf=5, total=  11.2s
[CV] max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=5, max_features=25, class_weight=balanced, min_samples_leaf=5, total=  11.0s
[CV] max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1, total=   9.8s
[CV] max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1, total=  10.3s
[CV] max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=15, max_features=5, class_weight=balanced, min_samples_leaf=1, total=  10.9s
[CV] max_depth=15, max_features=5, class_weight=bala

[CV]  max_depth=20, max_features=5, class_weight=balanced, min_samples_leaf=5, total=  10.6s
[CV] max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=20, max_features=5, class_weight=balanced, min_samples_leaf=5, total=  10.4s
[CV] max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=20, max_features=5, class_weight=balanced, min_samples_leaf=5, total=  10.7s
[CV] max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1 
[CV]  max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1, total=  29.2s
[CV] max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1, total=  30.5s
[CV] max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=3 
[CV]  max_depth=20, max_features=15, class_weight=balanced, min_samples_leaf=1, total=  30.4s
[CV] max_depth=20, max_features=15, class_we

[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  9.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=42, shuffle=False),
       error_score='raise',
       estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [5, 15, 20], 'max_features': [5, 15, 25], 'class_weight': ['balanced'], 'min_samples_leaf': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=2)

In [383]:
etcv.best_score_

0.7467941483460391

In [386]:
best_et = etcv.best_estimator_
best_et.fit(X_train_old, y_train_old)
res_et = best_et.predict(X_test_old)

yt = best_et.predict_proba(X_train_old)[:,1]
tr = Find_Optimal_Cutoff(y_train_old, yt)[0]

yt = np.where(yt < tr , 0, yt)
yt = np.where(yt >= tr , 1, yt)
print(np.sum(yt)/len(yt))


0.30327868852459017


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [387]:
res_et = np.where(res_et < tr , 0, res_et)
res_et = np.where(res_et >= tr , 1, res_et)
print(np.sum(res_et)/len(res_et))

0.2695631301008161


# Ensembling 2 

только на предиктах других моделей

(cat boost)

In [406]:
params4 = {'depth':[2, 3],
          'iterations':[150, 250],
          'learning_rate':[0.1], 
          'l2_leaf_reg':[7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}
cat_dims=[]

bestparams4 = catboost_param_tune(params4, X_tr, y_tr, cat_dims, 4, skf)




Borders generated
0:	learn 0.7050504635	total: 553ms	remaining: 2m 17s
1:	learn 0.7171150837	total: 574ms	remaining: 1m 11s
2:	learn 0.7344534937	total: 595ms	remaining: 49s
3:	learn 0.736625656	total: 616ms	remaining: 37.9s
4:	learn 0.7385765942	total: 638ms	remaining: 31.3s
5:	learn 0.7420435075	total: 659ms	remaining: 26.8s
6:	learn 0.7427540235	total: 680ms	remaining: 23.6s
7:	learn 0.7433234456	total: 701ms	remaining: 21.2s
8:	learn 0.7436393446	total: 724ms	remaining: 19.4s
9:	learn 0.7437690642	total: 762ms	remaining: 18.3s
10:	learn 0.7441273619	total: 792ms	remaining: 17.2s
11:	learn 0.7451180981	total: 813ms	remaining: 16.1s
12:	learn 0.7459551437	total: 835ms	remaining: 15.2s
13:	learn 0.7464080444	total: 857ms	remaining: 14.4s
14:	learn 0.7465098511	total: 879ms	remaining: 13.8s
15:	learn 0.7466482779	total: 900ms	remaining: 13.2s
16:	learn 0.7467373245	total: 922ms	remaining: 12.6s
17:	learn 0.7468280772	total: 964ms	remaining: 12.4s
18:	learn 0.7467608183	total: 986ms	rem

156:	learn 0.7517202846	total: 5.97s	remaining: 3.54s
157:	learn 0.7517205069	total: 6.03s	remaining: 3.51s
158:	learn 0.7517553685	total: 6.08s	remaining: 3.48s
159:	learn 0.7518263012	total: 6.13s	remaining: 3.44s
160:	learn 0.7518420226	total: 6.17s	remaining: 3.41s
161:	learn 0.7519092814	total: 6.22s	remaining: 3.38s
162:	learn 0.7519170996	total: 6.27s	remaining: 3.35s
163:	learn 0.7519366321	total: 6.32s	remaining: 3.31s
164:	learn 0.7519516018	total: 6.38s	remaining: 3.28s
165:	learn 0.7519822209	total: 6.42s	remaining: 3.25s
166:	learn 0.7520645213	total: 6.46s	remaining: 3.21s
167:	learn 0.7521796046	total: 6.49s	remaining: 3.17s
168:	learn 0.7522112959	total: 6.51s	remaining: 3.12s
169:	learn 0.7522146166	total: 6.53s	remaining: 3.07s
170:	learn 0.7522285273	total: 6.56s	remaining: 3.03s
171:	learn 0.7522633497	total: 6.58s	remaining: 2.99s
172:	learn 0.7523386883	total: 6.61s	remaining: 2.94s
173:	learn 0.7523528866	total: 6.63s	remaining: 2.9s
174:	learn 0.7523747854	total

66:	learn 0.7441479344	total: 3.11s	remaining: 8.48s
67:	learn 0.7441772511	total: 3.13s	remaining: 8.38s
68:	learn 0.7442227525	total: 3.15s	remaining: 8.27s
69:	learn 0.7442202359	total: 3.18s	remaining: 8.17s
70:	learn 0.7442199614	total: 3.2s	remaining: 8.07s
71:	learn 0.7442381789	total: 3.23s	remaining: 7.98s
72:	learn 0.7442508665	total: 3.25s	remaining: 7.88s
73:	learn 0.744272019	total: 3.27s	remaining: 7.79s
74:	learn 0.7443122454	total: 3.3s	remaining: 7.69s
75:	learn 0.7443542758	total: 3.33s	remaining: 7.61s
76:	learn 0.7443597666	total: 3.36s	remaining: 7.55s
77:	learn 0.744361793	total: 3.38s	remaining: 7.46s
78:	learn 0.7444135564	total: 3.43s	remaining: 7.43s
79:	learn 0.7444491287	total: 3.48s	remaining: 7.41s
80:	learn 0.7444903944	total: 3.54s	remaining: 7.38s
81:	learn 0.7445304377	total: 3.58s	remaining: 7.33s
82:	learn 0.7445738474	total: 3.63s	remaining: 7.3s
83:	learn 0.7445986147	total: 3.67s	remaining: 7.26s
84:	learn 0.7446363767	total: 3.72s	remaining: 7.23

221:	learn 0.7477331134	total: 8.64s	remaining: 1.09s
222:	learn 0.7477706532	total: 8.69s	remaining: 1.05s
223:	learn 0.7478220833	total: 8.71s	remaining: 1.01s
224:	learn 0.7478234952	total: 8.73s	remaining: 971ms
225:	learn 0.7478447262	total: 8.76s	remaining: 930ms
226:	learn 0.7478438176	total: 8.78s	remaining: 890ms
227:	learn 0.7478455694	total: 8.81s	remaining: 850ms
228:	learn 0.7478775334	total: 8.84s	remaining: 810ms
229:	learn 0.7478872338	total: 8.88s	remaining: 773ms
230:	learn 0.7478998756	total: 8.93s	remaining: 735ms
231:	learn 0.7479214922	total: 8.99s	remaining: 697ms
232:	learn 0.7479466124	total: 9.05s	remaining: 660ms
233:	learn 0.74796349	total: 9.1s	remaining: 622ms
234:	learn 0.7481026677	total: 9.15s	remaining: 584ms
235:	learn 0.7481055569	total: 9.2s	remaining: 546ms
236:	learn 0.7481080539	total: 9.25s	remaining: 507ms
237:	learn 0.7481161985	total: 9.3s	remaining: 469ms
238:	learn 0.7481211925	total: 9.35s	remaining: 430ms
239:	learn 0.7481462539	total: 9.

130:	learn 0.7535571827	total: 4.02s	remaining: 3.65s
131:	learn 0.753562961	total: 4.06s	remaining: 3.63s
132:	learn 0.7535751061	total: 4.1s	remaining: 3.61s
133:	learn 0.7535867216	total: 4.14s	remaining: 3.59s
134:	learn 0.7535911927	total: 4.17s	remaining: 3.55s
135:	learn 0.7535901468	total: 4.19s	remaining: 3.51s
136:	learn 0.7537170422	total: 4.22s	remaining: 3.48s
137:	learn 0.7537250038	total: 4.25s	remaining: 3.45s
138:	learn 0.753755203	total: 4.28s	remaining: 3.42s
139:	learn 0.7537574647	total: 4.3s	remaining: 3.38s
140:	learn 0.7537585302	total: 4.33s	remaining: 3.34s
141:	learn 0.7537614063	total: 4.35s	remaining: 3.31s
142:	learn 0.7538173271	total: 4.37s	remaining: 3.27s
143:	learn 0.7538272366	total: 4.4s	remaining: 3.24s
144:	learn 0.7538267529	total: 4.42s	remaining: 3.2s
145:	learn 0.7538276288	total: 4.45s	remaining: 3.17s
146:	learn 0.7538587169	total: 4.47s	remaining: 3.13s
147:	learn 0.753886746	total: 4.5s	remaining: 3.1s
148:	learn 0.7538856739	total: 4.52s	

...

Borders generated
0:	learn 0.6963266459	total: 603ms	remaining: 2m 30s
1:	learn 0.7112959494	total: 643ms	remaining: 1m 19s
2:	learn 0.7262497154	total: 684ms	remaining: 56.4s
3:	learn 0.728816038	total: 708ms	remaining: 43.5s
4:	learn 0.7342559371	total: 730ms	remaining: 35.8s
5:	learn 0.7357846956	total: 752ms	remaining: 30.6s
6:	learn 0.7379652073	total: 788ms	remaining: 27.3s
7:	learn 0.7390055103	total: 810ms	remaining: 24.5s
8:	learn 0.7392082174	total: 833ms	remaining: 22.3s
9:	learn 0.7394009301	total: 857ms	remaining: 20.6s
10:	learn 0.7398641803	total: 879ms	remaining: 19.1s
11:	learn 0.739688404	total: 902ms	remaining: 17.9s
12:	learn 0.7401314496	total: 926ms	remaining: 16.9s
13:	learn 0.7402314076	total: 951ms	remaining: 16s
14:	learn 0.7403086704	total: 988ms	remaining: 15.5s
15:	learn 0.740336438	total: 1.01s	remaining: 14.8s
16:	learn 0.7410413601	total: 1.04s	remaining: 14.2s
17:	learn 0.7411248131	total: 1.06s	remaining: 13.7s
18:	learn 0.7410830834	total: 1.08s	remai

155:	learn 0.7497208406	total: 4.84s	remaining: 2.92s
156:	learn 0.749745196	total: 4.88s	remaining: 2.89s
157:	learn 0.7499692583	total: 4.91s	remaining: 2.86s
158:	learn 0.7500205838	total: 4.93s	remaining: 2.82s
159:	learn 0.7500201066	total: 4.96s	remaining: 2.79s
160:	learn 0.7500319183	total: 4.98s	remaining: 2.75s
161:	learn 0.750074524	total: 5.01s	remaining: 2.72s
162:	learn 0.7501102597	total: 5.03s	remaining: 2.69s
163:	learn 0.750134772	total: 5.06s	remaining: 2.65s
164:	learn 0.7501980988	total: 5.09s	remaining: 2.62s
165:	learn 0.7502488622	total: 5.11s	remaining: 2.59s
166:	learn 0.7503355704	total: 5.14s	remaining: 2.55s
167:	learn 0.7504073556	total: 5.16s	remaining: 2.52s
168:	learn 0.7505185043	total: 5.18s	remaining: 2.48s
169:	learn 0.750626535	total: 5.21s	remaining: 2.45s
170:	learn 0.7506442754	total: 5.23s	remaining: 2.42s
171:	learn 0.7506559498	total: 5.26s	remaining: 2.38s
172:	learn 0.7506709317	total: 5.3s	remaining: 2.36s
173:	learn 0.7506873386	total: 5.

62:	learn 0.7534802532	total: 2.09s	remaining: 6.21s
63:	learn 0.753669253	total: 2.12s	remaining: 6.15s
64:	learn 0.7538187586	total: 2.14s	remaining: 6.09s
65:	learn 0.7538794249	total: 2.16s	remaining: 6.03s
66:	learn 0.7540101835	total: 2.19s	remaining: 5.97s
67:	learn 0.7540184327	total: 2.21s	remaining: 5.92s
68:	learn 0.754097212	total: 2.23s	remaining: 5.86s
69:	learn 0.7541177501	total: 2.26s	remaining: 5.82s
70:	learn 0.7541704679	total: 2.29s	remaining: 5.76s
71:	learn 0.7542004645	total: 2.32s	remaining: 5.73s
72:	learn 0.754285264	total: 2.35s	remaining: 5.69s
73:	learn 0.7543812478	total: 2.38s	remaining: 5.65s
74:	learn 0.7544087865	total: 2.4s	remaining: 5.6s
75:	learn 0.7544389923	total: 2.42s	remaining: 5.55s
76:	learn 0.7545361003	total: 2.45s	remaining: 5.5s
77:	learn 0.7545761502	total: 2.47s	remaining: 5.45s
78:	learn 0.7545903215	total: 2.5s	remaining: 5.4s
79:	learn 0.7546924301	total: 2.52s	remaining: 5.36s
80:	learn 0.7547101051	total: 2.56s	remaining: 5.34s
8

218:	learn 0.7590708475	total: 6.09s	remaining: 863ms
219:	learn 0.759071475	total: 6.13s	remaining: 836ms
220:	learn 0.7591052693	total: 6.15s	remaining: 807ms
221:	learn 0.7591296509	total: 6.17s	remaining: 779ms
222:	learn 0.7591481234	total: 6.2s	remaining: 750ms
223:	learn 0.7591629615	total: 6.22s	remaining: 722ms
224:	learn 0.7592061032	total: 6.25s	remaining: 695ms
225:	learn 0.7592325373	total: 6.28s	remaining: 667ms
226:	learn 0.7592336747	total: 6.3s	remaining: 639ms
227:	learn 0.7592418978	total: 6.34s	remaining: 612ms
228:	learn 0.7592527224	total: 6.36s	remaining: 583ms
229:	learn 0.7592654296	total: 6.38s	remaining: 555ms
230:	learn 0.7592784375	total: 6.41s	remaining: 527ms
231:	learn 0.7592778753	total: 6.43s	remaining: 499ms
232:	learn 0.759287275	total: 6.46s	remaining: 471ms
233:	learn 0.7593389535	total: 6.48s	remaining: 443ms
234:	learn 0.7593536478	total: 6.5s	remaining: 415ms
235:	learn 0.7593523143	total: 6.54s	remaining: 388ms
236:	learn 0.7594157522	total: 6.

124:	learn 0.7526664551	total: 3.7s	remaining: 3.7s
125:	learn 0.7527788704	total: 3.73s	remaining: 3.67s
126:	learn 0.7528229752	total: 3.76s	remaining: 3.64s
127:	learn 0.7528821824	total: 3.78s	remaining: 3.6s
128:	learn 0.7529122304	total: 3.8s	remaining: 3.57s
129:	learn 0.7529363512	total: 3.83s	remaining: 3.53s
130:	learn 0.752945193	total: 3.85s	remaining: 3.5s
131:	learn 0.7530863885	total: 3.88s	remaining: 3.46s
132:	learn 0.7532445358	total: 3.9s	remaining: 3.43s
133:	learn 0.7532762764	total: 3.94s	remaining: 3.41s
134:	learn 0.7532808378	total: 3.97s	remaining: 3.38s
135:	learn 0.7533309483	total: 3.99s	remaining: 3.35s
136:	learn 0.753347874	total: 4.01s	remaining: 3.31s
137:	learn 0.7533710341	total: 4.04s	remaining: 3.28s
138:	learn 0.7534086692	total: 4.06s	remaining: 3.24s
139:	learn 0.7535143927	total: 4.09s	remaining: 3.21s
140:	learn 0.753552727	total: 4.11s	remaining: 3.18s
141:	learn 0.7535603599	total: 4.14s	remaining: 3.15s
142:	learn 0.7535835984	total: 4.17s	

In [407]:
clf_cb4 = cb.CatBoostClassifier(**bestparams4)
clf_cb4.fit(X_tr, np.ravel(y_tr), cat_features = cat_dims)



In [417]:
res_cb4 = clf_cb4.predict_proba(X_te)[:,1]


yt = clf_cb4.predict_proba(X_tr)[:,1]
tr = Find_Optimal_Cutoff(y_tr, yt)[0]

res_cb4 = np.where(res_cb4 < tr , 0, res_cb4)
res_cb4 = np.where(res_cb4 >= tr , 1, res_cb4)
print(np.sum(res_cb4)/len(res_cb4))

0.37229956793086894


# correlation among all ensemble predictions 

In [414]:
# base = voting 
base=t_1
print('base - ens1_catb',matthews_corrcoef(base, res_cb3))
print('base - ens1_extr',matthews_corrcoef(base, res_et))
print('ens1_extr - ens1_catb',matthews_corrcoef(res_et, res_cb3))

#print('base - ens2_lr',matthews_corrcoef(base, y_lr))
print('base - ens2_catb',matthews_corrcoef(base, res_cb4))
#print('ens2_lr - ens2_catb',matthews_corrcoef(y_lr, res_cb4))

print('ens1_catb - ens2_catb',matthews_corrcoef(res_cb3, res_cb4))

print('ens1_extr - ens2_catb',matthews_corrcoef(res_et, res_cb4))
print('ens1_extr - ens2_catb',matthews_corrcoef(base, res_cb3))

base - ens1_catb 0.84642030854424
base - ens1_extr 0.8965252541591218
ens1_extr - ens1_catb 0.7825329198686126
base - ens2_catb 0.8507852823046459
ens1_catb - ens2_catb 0.9117924220053101
ens1_extr - ens2_catb 0.7776133680768467
ens1_extr - ens2_catb 0.84642030854424
